Hussein Hamie
500876254
AER 850 Machine Learning
Assignment 3

# Importing Libraries

In [ ]:
import pandas as pd
import sys
import scipy.io
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn
import numpy as np
import sklearn
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
np.set_printoptions(threshold=sys.maxsize)
tf.config.experimental.enable_tensor_float_32_execution(False)

# Loading in Data

In [ ]:
baselineDE = pd.read_excel('/mnt/d/AER 850 Machine Learning/baseline.xlsx',sheet_name="Baseline DE")

baseline007DE = pd.read_excel('/mnt/d/AER 850 Machine Learning/baseline.xlsx',sheet_name="0.007 DE")

baseline021DE = pd.read_excel('/mnt/d/AER 850 Machine Learning/baseline.xlsx',sheet_name="0.021 DE")

In [ ]:
baselineDE.head()

In [ ]:
baselineDE.insert(0, 'Fault', 0)
baselineDE = baselineDE.rename(columns={'Baseline 0HP DE X97':'0HP','Baseline 1HP DE X98':'1HP','Baseline 2HP DE X99':'2HP','Baseline 3HP DE':'3HP'})
baselineDE.head()

In [5]:
baseline007DE.insert(0,'Fault',1)
baseline007DE = baseline007DE.rename(columns={'12K 0.007" 0HP':'0HP','12K 0.007" 1HP':'1HP','12K 0.007" 2HP':'2HP','12K 0.007" 3HP':'3HP'})
baseline007DE.head()

,Fault,0HP,1HP,2HP,3HP
0,1,-0.083004,-0.277602,-0.093238,0.222699
1,1,-0.195734,-0.044345,0.187288,0.093238
2,1,0.233419,0.117603,0.217663,-0.146516
3,1,0.103958,-0.145055,0.070172,0.177217
4,1,-0.181115,-0.111430,0.100385,0.248526


In [6]:
baseline021DE.insert(0,'Fault',1)
baseline021DE = baseline021DE.rename(columns={'12K 0.021" 0HP':'0HP','12K 0.021" 1HP':'1HP','12K 0.021" 2HP':'2HP','12K 0.021" 3HP':'3HP'})
baseline021DE.head()

,Fault,0HP,1HP,2HP,3HP
0,1,1.189431,0.171369,-0.402027,-0.206293
1,1,-0.177866,0.117765,0.548219,-0.007310
2,1,-0.774816,-0.097055,0.931565,0.222536
3,1,0.501518,0.009746,-0.218881,-0.005685
4,1,0.993697,0.060913,-1.079788,0.031675


In [7]:
faultdata = pd.concat([baselineDE,baseline007DE,baseline021DE])
faultdata.head()

,Fault,0HP,1HP,2HP,3HP
0,0,0.053197,0.145667,0.064254,0.014603
1,0,0.088662,0.097796,0.063002,0.054449
2,0,0.099718,0.054856,-0.004381,0.107646
3,0,0.058621,0.036982,-0.035882,0.133722
4,0,-0.004590,0.054445,-0.023991,0.112652


In [8]:
faultdatashuffled = faultdata.sample(frac=1,random_state = 42)
faultdatashuffled.head()

,Fault,0HP,1HP,2HP,3HP
119391,1,0.053604,-0.040203,-0.719994,0.049543
195186,0,0.005424,0.074991,0.117242,-0.029415
41740,1,-0.099085,-0.136608,0.027289,0.029076
155158,0,-0.048607,-0.028558,-0.045895,-0.136434
102538,0,0.001043,-0.000411,0.017315,0.046313


In [9]:
def plot_columns(dataframe_cols):
    fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(16, 9), sharey=True)
    colors = plt.cm.get_cmap('Set1', len(axes[0]))
    for i, ax in enumerate(axes.flatten()):
        if i < len(dataframe_cols):
            col = dataframe_cols[i]
            ax.scatter(col.index, col.values, s=0.05, color=colors(i % len(axes[0])))
            ax.set_title(col.name)
        else:
            ax.set_visible(False)
    for i, ax in enumerate(axes[:, 0]):
        if i == 0:
            ax.set_ylabel('normal', rotation=0, fontsize='large', labelpad=40)
        elif i == 1:
            ax.set_ylabel('0.0071 fault', rotation=0, fontsize='large', labelpad=40)
        elif i == 2:
            ax.set_ylabel('0.021 fault', rotation=0, fontsize='large', labelpad=40)
    plt.tight_layout()
    plt.show()

In [2]:
listsofdatas= [baselineDE['0HP'],baselineDE['1HP'],baselineDE['2HP'],baselineDE['3HP'],baseline007DE['0HP'],baseline007DE['1HP'],baseline007DE['2HP'],baseline007DE['3HP'],baseline021DE['0HP'],baseline021DE['1HP'],baseline021DE['2HP'],baseline021DE['3HP']]
plot_columns(listsofdatas)

NameError: name 'baselineDE' is not defined

In [ ]:
def transpose_and_add_fault_column(df,fault):
    # drop 'Fault' column (if it exists)
    if 'Fault' in df.columns:
        df = df.drop(columns=['Fault'])

    # transpose DataFrame
    df = pd.DataFrame(df.values.T, index=df.columns, columns=df.index)

    # add 'Fault' column with all values set to the fault
    # df.insert(0, 'Fault', fault)

    # return the modified DataFrame
    return df

In [ ]:
trans_baselineDE=transpose_and_add_fault_column(baselineDE,0)
trans_baseline021DE=transpose_and_add_fault_column(baseline021DE,1)
trans_baseline007DE=transpose_and_add_fault_column(baseline007DE,1)

In [ ]:
trans_baseline021DE


In [ ]:
df_transposed = trans_baselineDE.T
df_transposed

In [ ]:
chunk_size = 10

if len(df_transposed) >= chunk_size:
    # Split the data frame into equal chunks using numpy.array_split()
    df_chunks = np.array_split(df_transposed, len(df_transposed) // chunk_size, axis=0)

    # Concatenate the chunks into a new data frame with new columns
    new_df = pd.concat(df_chunks, axis=1)

    # Print the new data frame
    print(new_df)

In [ ]:
df = pd.concat([trans_baselineDE, trans_baseline007DE, trans_baseline021DE],axis=0)
df

In [ ]:
mask = df.isna()
first_nan_index = mask.any(axis=0).idxmax()
print(first_nan_index)

In [ ]:
data=df
data

Now the NAN's are removed we can break the data into smaller 1000 chunks this way we can do a train test split with more than the 12 tests we have. This is possible because the data is relatively uniform over the course of time in each test


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
len(X_train)


In [ ]:
X_train

In [ ]:
tf.config.experimental.enable_tensor_float_32_execution(False)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
model.evaluate(X_test, y_test)